# Sample data from each well
Deterministically sample data from each plate dataset and get the intersection of all features not containing nans.

### Import Libraries

In [ ]:
import json
import pathlib
import sys

import pandas as pd

sys.path.append(str(pathlib.Path.cwd().parent / "utils"))
from DeterministicSampling import DeterministicSampling

## Define paths

### Inputs

In [1]:
# Plate morphology data
sc_data_path = pathlib.Path(sys.argv[1]).resolve(strict=True)
sc_data_dir_name = sc_data_path.parent.name
plate_path = pathlib.Path(sc_data_path).resolve(strict=True)
platedf = pd.read_parquet(plate_path)

sampled_plate_jump_path = pathlib.Path(sys.argv[2])

anomaly_data_path = pathlib.Path(sys.argv[3]) / sc_data_dir_name
anomaly_data_path.mkdir(parents=True, exist_ok=True)
feat_col_path = anomaly_data_path / "feature_columns.json"

--------------------------------------------------------------
FileNotFoundError            Traceback (most recent call last)
Cell In[5], line 2
      1 # Plate morphology data
----> 2 plate_path = pathlib.Path(sys.argv[1]).resolve(strict=True)
      3 platedf = pd.read_parquet(plate_path)
      5 # Boolean flag for if the data is single-cell

File ~/mambaforge-pypy3/envs/jump_sc/lib/python3.9/pathlib.py:1215, in Path.resolve(self, strict)
   1209 def resolve(self, strict=False):
   1210     """
   1211     Make the path absolute, resolving all symlinks on the way and also
   1212     normalizing it (for example turning slashes into backslashes under
   1213     Windows).
   1214     """
-> 1215     s = self._flavour.resolve(self, strict=strict)
   1216     if s is None:
   1217         # No symlink resolution => for consistency, raise an error if
   1218         # the path doesn't exist or is forbidden
   1219         self.stat()

File ~/mambaforge-pypy3/envs/jump_sc/lib/python3.9/pat

### Outputs

In [2]:
plate_data_path = sampled_plate_jump_path / f"{plate_path.parent.name}.parquet"

--------------------------------------------------------------
NameError                    Traceback (most recent call last)
Cell In[7], line 1
----> 1 plate_data_path = sampled_plate_jump_path / plate_path.parent.name

NameError: name 'sampled_plate_jump_path' is not defined


## Save feature columns that don't contain NaNs

In [ ]:
feat_cols = []
non_na_cols = platedf.columns[platedf.notna().all()].tolist()
non_na_cols = [col for col in non_na_cols if not "Metadata" in col]

if feat_col_path.exists():
    with feat_col_path.open("r") as feat_col_obj:
        feat_cols = json.load(feat_col_obj)
        feat_cols = list(set(non_na_cols) & set(feat_cols))

else:
    feat_cols = non_na_cols

with feat_col_path.open("w") as feat_col_obj:
    json.dump(feat_cols, feat_col_obj)

## Sample single-cell data by Hash

In [3]:
ds = DeterministicSampling(
    _platedf=platedf,
    # Number of samples per plate needed to train the JUMP isolation forests
    # See identify_anomalous_single_cells_fs.py for more details
    # Only 4_000 are needed, however each sampling will likely not be exactly 4_000 samples
    # due to the probabilistic nature of hash-based sampling.
    # To ensure a sufficient sample size I increased the threshold.
    _samples_per_plate=4_100,
    _plate_column="Metadata_Plate",
    _well_column="Metadata_Well",
    _cell_id_columns=["Metadata_Site", "Metadata_ObjectNumber"],
)

sampled_platedf = ds.sample_plate_deterministically(_sample_strategy="well_sampling")

--------------------------------------------------------------
NameError                    Traceback (most recent call last)
Cell In[8], line 7
      4 hash_cols = ["Metadata_Plate", "Metadata_Well"]
      5 group_cols = hash_cols.copy()
----> 7 if is_sc:
      8     hash_cols += ["Metadata_Site", "Metadata_ObjectNumber"]
     10 platedf["Metadata_farmhash"] = platedf.apply(
     11     lambda row: Fingerprint64("".join(str(row[col]) for col in hash_cols)) % divisor,
     12     axis=1,
     13 )

NameError: name 'is_sc' is not defined


## Concatenate and Save Sampled Plate data

In [4]:
if plate_data_path.exists():
    sampled_platedf = pd.concat(
        [sampled_platedf, pd.read_parquet(plate_data_path)], axis=0
    )

sampled_platedf.to_parquet(plate_data_path)

--------------------------------------------------------------
NameError                    Traceback (most recent call last)
Cell In[12], line 1
----> 1 if plate_data_path.exists():
      2     sampled_platedf = pd.concat(
      3         [sampled_platedf, pd.read_parquet(plate_data_path)], axis=0
      4     )
      6 sampled_platedf.to_parquet(plate_data_path)

NameError: name 'plate_data_path' is not defined
